<a href="https://colab.research.google.com/github/francoriboratig/TP2-Datos/blob/master/Arboles_de_decision.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
import pandas as pd
import tensorflow as tf
import numpy as np
from google.colab import files
from sklearn.metrics import precision_score, recall_score, accuracy_score
!pip install -q tensorflow-hub
import tensorflow_hub as hub
from sklearn.model_selection import train_test_split

urltrain = "https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/train-features.csv"
urltest = "https://raw.githubusercontent.com/francoriboratig/TP2-Datos/master/test-features.csv"

train = pd.read_csv(urltrain)
test = pd.read_csv(urltest)

train.head()

,Unnamed: 0,id,keyword,location,text,target,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1,69,13,0,0,0,0,0,False,0,0,False,7,False,False
1,1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1,38,7,1,0,0,0,0,False,0,0,False,3,True,False
2,2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1,133,22,0,0,0,0,0,False,0,0,False,13,False,False
3,3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1,65,8,1,0,0,0,0,False,0,0,False,2,False,False
4,4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1,88,16,1,0,0,0,0,False,0,0,False,10,False,False


In [16]:
test.head()

,Unnamed: 0,id,keyword,location,text,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,0,0,NaN,NaN,Just happened a terrible car crash,34,6,0,0,0,0,0,False,0,0,False,4,False,False
1,1,2,NaN,NaN,"Heard about #earthquake is different cities, s...",64,9,0,0,0,0,0,False,0,0,False,7,False,False
2,2,3,NaN,NaN,"there is a forest fire at spot pond, geese are...",96,19,1,0,0,0,0,False,0,0,False,10,False,False
3,3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires,40,4,1,0,0,0,0,False,0,0,False,3,False,True
4,4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan,45,8,0,0,0,0,0,False,0,0,False,6,True,False


In [17]:
train.columns

Index(['Unnamed: 0', 'id', 'keyword', 'location', 'text', 'target',
       'character_count', 'word_count', 'contains_word_fire',
       'contains_word_storm', 'contains_word_flood', 'contains_word_death',
       'contains_word_love', 'has_mentions', 'amount_of_exclamation_marks',
       'amount_of_interrogation_marks', 'has_link', 'stopwords_count',
       'contains_countries', 'contains_cities'],
      dtype='object')

In [18]:
y_train = train['target']

# HACEMOS UN EMBEDDING PROVISTO POR TensorFlow Hub, por fuera del modelo, de esa forma puedo agregarle features al array

embedding_url = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

hub_layer = hub.KerasLayer(embedding_url,
                           input_shape = [], 
                           dtype = tf.string,
                           trainable = True)

train_text_array = np.asarray(train['text'])
train_text_embedded = hub_layer(train_text_array)
train_text_embedded_array = train_text_embedded.numpy()
train_text_embedded_df = pd.DataFrame(data = train_text_embedded_array)
test_text_array = np.asarray(test['text'])
test_text_embedded = hub_layer(test_text_array)
test_text_embedded_array = train_text_embedded.numpy()
test_text_embedded_df = pd.DataFrame(data = train_text_embedded_array)


x_train = pd.concat([train_text_embedded_df, train.drop(['Unnamed: 0','id','location','text','target','keyword'], axis = 1)], axis=1)
x_test = pd.concat([test_text_embedded_df, test.drop(['Unnamed: 0','id','location','text','keyword'], axis = 1)], axis=1)

#x_train = train.drop(['Unnamed: 0','id','location','text','target','keyword'], axis = 1)
#x_test = test.drop(['Unnamed: 0','id','location','text','keyword'], axis = 1)
x_train.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,character_count,word_count,contains_word_fire,contains_word_storm,contains_word_flood,contains_word_death,contains_word_love,has_mentions,amount_of_exclamation_marks,amount_of_interrogation_marks,has_link,stopwords_count,contains_countries,contains_cities
0,0.007239,0.156896,1.295307,-0.040780,-0.160369,-1.200443,-0.404513,-0.523973,-0.235587,-0.350222,-0.289882,-0.471979,-0.076219,0.170489,-0.832377,0.022601,0.161255,-0.640868,-0.569074,0.040001,69,13,0,0,0,0,0,False,0,0,False,7,False,False
1,-0.498373,1.282888,-0.188361,-0.947347,0.880581,-0.178855,-0.015432,0.383270,-1.508848,1.753281,-0.754933,0.601241,-0.143784,-0.891944,0.432137,1.346741,-1.485925,1.576216,-0.378781,0.942368,38,7,1,0,0,0,0,False,0,0,False,3,True,False
2,0.088144,-0.311402,0.604415,1.713081,1.320312,-3.201303,0.844513,0.962179,-1.535314,2.206953,-0.279025,0.010960,-2.228273,0.204312,-2.175010,1.344161,0.118840,-1.238801,-0.255011,0.695272,133,22,0,0,0,0,0,False,0,0,False,13,False,False
3,-1.083202,0.694350,-0.293344,0.301429,1.887291,-2.864778,-0.018864,0.176138,-2.388204,1.020881,0.144147,0.802330,-1.326284,-0.055402,-0.268148,1.400147,-1.432997,-1.546905,0.456249,0.117063,65,8,1,0,0,0,0,False,0,0,False,2,False,False
4,0.296060,-0.401119,0.260654,-0.133400,0.043096,-0.781329,-0.456910,0.952776,0.359138,0.342683,-1.734295,1.611835,-0.623508,-0.758112,-0.305311,1.009357,-0.759674,-0.749319,-1.216560,0.413135,88,16,1,0,0,0,0,False,0,0,False,10,False,False


In [19]:
x_train = x_train*1 #Pongo todos los bool en numeros
x_test = x_test*1

In [20]:
# Saving feature names for later use
feature_list = list(x_train.columns)
# Convert to numpy array
features = np.array(x_train)

labels = np.array(y_train)

In [21]:
from sklearn.model_selection import train_test_split

# Split the data into training and testing sets
train_features, val_features, train_labels, val_labels = train_test_split(features, labels, test_size = 0.25, random_state = 70)

In [22]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators = 5000,criterion='gini',max_depth=100, random_state = 92)

rf.fit(train_features, train_labels)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=100, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=5000,
                       n_jobs=None, oob_score=False, random_state=92, verbose=0,
                       warm_start=False)

In [23]:
# Use the forest's predict method on the test data
preds = rf.predict(val_features)

print (accuracy_score(val_labels,preds))

0.7862394957983193


Aqui adelante, uso xgboost

In [24]:
import xgboost as xgb

In [25]:
model = xgb.XGBClassifier(eta=0.3,max_depth=4,objective='reg:squarederror',gamma=0,colsample_bytree=1,min_child_weight=2.5,subsample=0.7)

In [26]:
model.fit(train_features,train_labels)

preds = model.predict(val_features)
print (accuracy_score(val_labels,preds))

0.779936974789916


Lgbm lets go!

In [27]:
import lightgbm as lgb

In [28]:
lgb_train = lgb.Dataset(train_features, train_labels)
lgb_eval = lgb.Dataset(val_features,val_labels, reference = lgb_train)

params = {
    'objective':'binary',
    'metric' : 'xentropy',
    'num_leaves' : 30, #Just for trysies
    'learning_rate' : 0.05,
    'feature_fraction': 0.9,
}

gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)

preds = gbm.predict(val_features, num_iteration=gbm.best_iteration)
#print (accuracy_score(val_labels,preds))

[1]	valid_0's xentropy: 0.668424
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's xentropy: 0.654401
[3]	valid_0's xentropy: 0.642293
[4]	valid_0's xentropy: 0.631122
[5]	valid_0's xentropy: 0.620877
[6]	valid_0's xentropy: 0.611165
[7]	valid_0's xentropy: 0.603031
[8]	valid_0's xentropy: 0.595522
[9]	valid_0's xentropy: 0.588395
[10]	valid_0's xentropy: 0.58197
[11]	valid_0's xentropy: 0.57593
[12]	valid_0's xentropy: 0.569914
[13]	valid_0's xentropy: 0.564906
[14]	valid_0's xentropy: 0.559955
[15]	valid_0's xentropy: 0.555298
[16]	valid_0's xentropy: 0.551361
[17]	valid_0's xentropy: 0.546272
[18]	valid_0's xentropy: 0.542452
[19]	valid_0's xentropy: 0.539
[20]	valid_0's xentropy: 0.535494
[21]	valid_0's xentropy: 0.532489
[22]	valid_0's xentropy: 0.529409
[23]	valid_0's xentropy: 0.526336
[24]	valid_0's xentropy: 0.5229
[25]	valid_0's xentropy: 0.52017
[26]	valid_0's xentropy: 0.517768
[27]	valid_0's xentropy: 0.515657
[28]	valid_0's xentropy: 0.513221
[29

CatBoost

In [29]:
!pip install catboost

In [30]:
from catboost import CatBoostClassifier

model = CatBoostClassifier()
model.fit(train_features,train_labels)
preds = model.predict(val_features)

print (accuracy_score(val_labels,preds))

Learning rate set to 0.021676
0:	learn: 0.6857212	total: 8.97ms	remaining: 8.96s
1:	learn: 0.6782181	total: 17.4ms	remaining: 8.67s
2:	learn: 0.6720993	total: 25.5ms	remaining: 8.48s
3:	learn: 0.6658663	total: 33.8ms	remaining: 8.42s
4:	learn: 0.6600861	total: 42.3ms	remaining: 8.42s
5:	learn: 0.6543150	total: 50.7ms	remaining: 8.4s
6:	learn: 0.6476697	total: 59.2ms	remaining: 8.39s
7:	learn: 0.6417092	total: 69.2ms	remaining: 8.58s
8:	learn: 0.6369015	total: 77.3ms	remaining: 8.51s
9:	learn: 0.6315444	total: 85.6ms	remaining: 8.47s
10:	learn: 0.6268151	total: 93.8ms	remaining: 8.43s
11:	learn: 0.6223582	total: 102ms	remaining: 8.4s
12:	learn: 0.6181098	total: 110ms	remaining: 8.38s
13:	learn: 0.6137277	total: 119ms	remaining: 8.35s
14:	learn: 0.6096199	total: 127ms	remaining: 8.33s
15:	learn: 0.6061958	total: 135ms	remaining: 8.31s
16:	learn: 0.6021849	total: 144ms	remaining: 8.3s
17:	learn: 0.5984520	total: 153ms	remaining: 8.35s
18:	learn: 0.5950559	total: 162ms	remaining: 8.35s
19: